![example](images/director_shot.jpeg)

# Microsoft Movie Studio Market Analysis & Recommendations

**Author:** Kristin Cooper
***

## Overview

A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***
Questions to consider:
* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?
***

## Data Understanding

Describe the data being used for this project.
***
Questions to consider:
* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?
***

Files:
* tn.movie_budgets - production budget, domestic gross, worldwide gross
* tmdb.movies - genre, popularity
* imdb.title.basics - year, runtime, genre

Process Notes:
* Chose to leverage SQLite database - merging dataframes would be difficult because there is not a common unique identifier across all 3 of my data sources

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

%matplotlib inline

In [2]:
def clean_money_columns(df, column):
    df[column] = df[column].apply([lambda x: x[1:]])
    df[column] = df[column].apply([lambda x: x.replace(',', '')])
    df[column] = df[column].apply([lambda x: int(x)])

In [3]:
df_moviebudgets = pd.DataFrame(pd.read_csv('data/zippedData/tn.movie_budgets.csv'))
df_moviebudgets.isna().any() # checked to make sure no null values
df_moviebudgets.duplicated('movie') # checked for duplicate rows based on "movie" column
df_moviebudgets.drop(columns='id', inplace=True) # dropped ID column because there were duplicate values
df_moviebudgets['release_year'] = df_moviebudgets['release_date'].apply(lambda x: x[-4:]) # created a column for release year to simplify/group
df_moviebudgets.sort_values('release_year', inplace=True)
clean_money_columns(df=df_moviebudgets, column='production_budget')
clean_money_columns(df=df_moviebudgets, column='domestic_gross')
clean_money_columns(df=df_moviebudgets, column='worldwide_gross')
df_moviebudgets['release_year'] = df_moviebudgets['release_year'].astype(int)

print(df_moviebudgets.info())
df_moviebudgets

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5782 entries, 5677 to 535
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   release_date       5782 non-null   object
 1   movie              5782 non-null   object
 2   production_budget  5782 non-null   int64 
 3   domestic_gross     5782 non-null   int64 
 4   worldwide_gross    5782 non-null   int64 
 5   release_year       5782 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 316.2+ KB
None


,release_date,movie,production_budget,domestic_gross,worldwide_gross,release_year
5677,"Feb 8, 1915",The Birth of a Nation,110000,10000000,11000000,1915
5523,"Sep 5, 1916",Intolerance,385907,0,0,1916
5614,"Dec 24, 1916","20,000 Leagues Under the Sea",200000,8000000,8000000,1916
5683,"Sep 17, 1920",Over the Hill to the Poorhouse,100000,3000000,3000000,1920
4569,"Dec 30, 1925",Ben-Hur: A Tale of the Christ,3900000,9000000,9000000,1925
...,...,...,...,...,...,...
2292,"Jan 25, 2019",Serenity,25000000,8547045,11367029,2019
4135,"Feb 8, 2019",The Prodigy,6000000,14856291,19789712,2019
1205,"Dec 31, 2020",Hannibal the Conqueror,50000000,0,0,2020
194,"Dec 31, 2020",Moonfall,150000000,0,0,2020


In [11]:
df_imdbbasics = pd.DataFrame(pd.read_csv('data/zippedData/imdb.title.basics.csv'))
df_imdbbasics.drop(columns='tconst', inplace=True)
df_imdbbasics['genres'] = df_imdbbasics['genres'].fillna('n/a')
df_imdbbasics['original_title'] = df_imdbbasics['original_title'].fillna(df_imdbbasics['primary_title'])
df_imdbbasics['runtime_minutes'] = df_imdbbasics['runtime_minutes'].fillna('0').astype(int)

for index, string in enumerate(df_imdbbasics['genres']):
    df_imdbbasics['genres'][index] = string.replace(',', ', ')
    
print(df_imdbbasics.info())
df_imdbbasics

<ipython-input-11-6502eb36dd1f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_imdbbasics['genres'][index] = string.replace(',', ', ')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   primary_title    146144 non-null  object
 1   original_title   146144 non-null  object
 2   start_year       146144 non-null  int64 
 3   runtime_minutes  146144 non-null  int64 
 4   genres           146144 non-null  object
dtypes: int64(2), object(3)
memory usage: 5.6+ MB
None


,primary_title,original_title,start_year,runtime_minutes,genres
0,Sunghursh,Sunghursh,2013,175,"Action, Crime, Drama"
1,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114,"Biography, Drama"
2,The Other Side of the Wind,The Other Side of the Wind,2018,122,Drama
3,Sabse Bada Sukh,Sabse Bada Sukh,2018,0,"Comedy, Drama"
4,The Wandering Soap Opera,La Telenovela Errante,2017,80,"Comedy, Drama, Fantasy"
...,...,...,...,...,...
146139,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,2019,123,Drama
146140,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,2015,0,Documentary
146141,Dankyavar Danka,Dankyavar Danka,2013,0,Comedy
146142,6 Gunn,6 Gunn,2017,116,n/a


In [5]:
df_tmdbmovies = pd.DataFrame(pd.read_csv('data/zippedData/tmdb.movies.csv'))
df_tmdbmovies.isna().any()
df_tmdbmovies.drop(columns=['Unnamed: 0','genre_ids', 'id'], inplace=True)

print(df_tmdbmovies.info())
df_tmdbmovies

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26517 entries, 0 to 26516
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   original_language  26517 non-null  object 
 1   original_title     26517 non-null  object 
 2   popularity         26517 non-null  float64
 3   release_date       26517 non-null  object 
 4   title              26517 non-null  object 
 5   vote_average       26517 non-null  float64
 6   vote_count         26517 non-null  int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 1.4+ MB
None


,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,en,Inception,27.920,2010-07-16,Inception,8.3,22186
...,...,...,...,...,...,...,...
26512,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1
26513,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1
26514,en,The Last One,0.600,2018-10-01,The Last One,0.0,1
26515,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1


In [12]:
import sqlite3
conn = sqlite3.connect('movies.db')
cur = conn.cursor()


df_moviebudgets.to_sql('budgets', con=conn, if_exists='replace', index=True)
df_imdbbasics.to_sql('genres', con=conn, if_exists='replace', index=True)
df_tmdbmovies.to_sql('ratings', con=conn, if_exists='replace', index=True)

In [13]:
def query_to_df(query):
    cur.execute(query)
    df = pd.DataFrame(cur.fetchall())
    df.columns = [x[0] for x in cur.description]
    return df

In [14]:
df_tmdbmovies['vote_count'].describe()

count    26517.000000
mean       194.224837
std        960.961095
min          1.000000
25%          2.000000
50%          5.000000
75%         28.000000
max      22186.000000
Name: vote_count, dtype: float64

In [52]:
query1 = '''SELECT b.movie AS b_title, r.title AS r_title, g.primary_title AS g_title, 
                    b.release_year, r.original_language, b.production_budget, b.worldwide_gross, r.vote_average, 
                    r.vote_count, r.popularity, g.runtime_minutes, g.genres
            FROM budgets b
            JOIN ratings r ON b.movie = r.title
            JOIN genres g ON b.movie = g.primary_title
            WHERE b.worldwide_gross > 0
            ORDER BY worldwide_gross DESC
            ;'''
df1 = query_to_df(query1)
df2 = df1.drop_duplicates(subset=['b_title', 'release_year', 'worldwide_gross'])

In [58]:
top_100 = df2.sort_values('worldwide_gross', ascending=False)[0:100]
top_100[top_100.duplicated(subset=['b_title', 'release_year'])] #checked for no duplicates in top 100
top_100

,b_title,r_title,g_title,release_year,original_language,production_budget,worldwide_gross,vote_average,vote_count,popularity,runtime_minutes,genres
0,Avatar,Avatar,Avatar,2009,en,425000000,2776345279,7.4,18676,26.526,93,Horror
1,Avengers: Infinity War,Avengers: Infinity War,Avengers: Infinity War,2018,en,300000000,2048134200,8.3,13948,80.773,149,"Action, Adventure, Sci-Fi"
2,Jurassic World,Jurassic World,Jurassic World,2015,en,215000000,1648854864,6.6,14056,20.709,124,"Action, Adventure, Sci-Fi"
3,Furious 7,Furious 7,Furious 7,2015,en,190000000,1518722794,7.3,6538,20.396,137,"Action, Crime, Thriller"
4,The Avengers,The Avengers,The Avengers,2012,en,225000000,1517935897,7.6,19673,50.289,143,"Action, Adventure, Sci-Fi"
...,...,...,...,...,...,...,...,...,...,...,...,...
169,Kong: Skull Island,Kong: Skull Island,Kong: Skull Island,2017,en,185000000,561072059,6.3,6054,29.388,118,"Action, Adventure, Fantasy"
170,Cars 2,Cars 2,Cars 2,2011,en,200000000,560155383,5.9,4058,23.498,106,"Adventure, Animation, Comedy"
171,Ted,Ted,Ted,2012,en,50000000,556016627,6.3,7727,14.475,106,"Comedy, Fantasy"
172,Puss in Boots,Puss in Boots,Puss in Boots,2011,en,130000000,554987477,6.3,1422,9.078,90,"Action, Adventure, Animation"


## code bannk


In [ ]:
# code bank

# import json
# f = open('tmdb_genre_ids.json') 
# data = json.load(f)
# df_genreids = pd.DataFrame(data['genres'])
# keys = list(df_genreids['id'])
# values = list(df_genreids['name'])
# genre_dict = dict(zip(keys, values))

# df_tmdbmovies['genre_ids_list'] = df_tmdbmovies['genre_ids'].apply([lambda x: x.strip('[]').split(', ')])

# def str_to_int(str_list):
#     int_list=[]
#     for item in str_list:
#         int_list.append(int(item))
        
# df_tmdbmovies['genre_ids_list'].apply(str_to_int)

# for index in range(len(df_tmdbmovies)):
#     for i in df_tmdbmovies['genre_ids_list'][index]:
#         int(i)

## Data Preparation

Describe and justify the process for preparing the data for analysis.

***
Questions to consider:
* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to clean the data

## Data Modeling
Describe and justify the process for analyzing or modeling the data.

***
Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?
***

In [ ]:
# Here you run your code to model the data


## Evaluation
Evaluate how well your work solves the stated business problem.

***
Questions to consider:
* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?
***

## Conclusions
Provide your conclusions about the work you've done, including any limitations or next steps.

***
Questions to consider:
* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?
***